# CW1 - Object Recognition using CNN
To apply machine learning alorithms to clasify the testing images into object categories. Then use a model to perform classification and report quantitative results.

Due: Monday 19th April

# Introduction

The aim is to evaluate the use of CNN's in image recognition and the affect of adding multiple layers.

# Imports 

In [ ]:
import numpy as np
from numpy import load
from numpy import asarray
from numpy import save
import matplotlib.pyplot as plt
import tensorflow as tf
import skimage.feature
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D


from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LearningRateScheduler


# Load Data

CIFAR-100 dataset
Each category 
contains 500 training images and 100 testing images, which are stored in two 4D arrays.

The size of each image is fixed at 32x32x3, corresponding to height, width, and colour 
channel, respectively.

There are 100 different categories of objects
each has 500 images for training and 100 images for testing.
Split the data into train and test sections

In [ ]:
images = np.load('trnImage.npy')
label_fine = np.load('trnLabel_fine.npy')
label_coarse = np.load('trnLabel_coarse.npy')

#image_index = 1 # pick a specific image
#image = images[:, :, :, image_index]

test_images = np.load('tstImage.npy')
test_label_fine = np.load('tstLabel_fine.npy')
test_label_coarse = np.load('tstLabel_coarse.npy')

print(f'Images Shape: {images.shape}')
print(f'Train Labels Fine Shape: {label_fine.shape}')
print(f'Train Labels Coarse Shape: {label_coarse.shape}')

## Shuffle data to ensure not ordered

In [ ]:
images = shuffle(images, random_state=0)
label_fine, label_coarse = shuffle(label_fine, label_coarse, random_state=0) #make sure the samples are not ordered


test_images = shuffle(test_images, random_state=0)
test_label_fine, test_label_coarse = shuffle(test_label_fine, test_label_coarse, random_state=0)

# CNN

## Normalise the data, for each image do a hog, add how to array, train images

In [ ]:
train_images_reshaped = np.transpose(images, (3, 0, 1, 2))
test_images_reshaped = np.transpose(test_images, (3, 0, 1, 2))

train_images_normalised = np.divide(train_images_reshaped, 255)
test_images_normalised = np.divide(test_images_reshaped, 255)

label_coarse_reshaped = np.expand_dims(label_coarse, axis=1)
test_label_coarse_reshaped = np.expand_dims(test_label_coarse, axis=1)



In [ ]:
print((train_images_normalised.shape,label_coarse_reshaped.shape))
print((test_images_normalised.shape,test_label_coarse_reshaped.shape))


In [ ]:
xTrn_split, xTst_split, yTrn_split, yTst_split = train_test_split(train_images_normalised, label_coarse_reshaped, test_size=0.2)

In [ ]:
#Dimension of the CIFAR10 dataset
print((xTrn_split.shape, yTrn_split.shape))
print((xTst_split.shape, yTst_split.shape))
print((test_images_normalised.shape, test_label_coarse_reshaped.shape))

#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 20 coarse classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 200
yTrn_split=to_categorical(yTrn_split)
yTst_split=to_categorical(yTst_split)
test_label_coarse_reshaped=to_categorical(test_label_coarse_reshaped)

#Verifying the dimension after one hot encoding
print((xTrn_split.shape,yTrn_split.shape))
print((xTst_split.shape,yTst_split.shape))
print((test_images_normalised.shape,test_label_coarse_reshaped.shape))


#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

ytrntst_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(xTrn_split)
ytrntst_generator.fit(xTst_split)
test_generator.fit(test_images_normalised)

## Check the Data cardinality

## Split Training data into a train and test split

## Create a Tensorflow Convolutional Neural Network on the training set using Conv2D and pooling Layers
AlexNet

https://analyticsindiamag.com/hands-on-guide-to-implementing-alexnet-with-keras-for-multi-class-image-classification/#:~:text=Implementing%20in%20Keras,it%20a%20pre%2Dtrained%20model.

https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98

In [1]:
#Importing library
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

np.random.seed(1000)

#Instantiation
alex_net = Sequential()

#1st Convolutional Layer
alex_net.add(Conv2D(filters=96, input_shape=(32,32,3), kernel_size=(11,11), strides=(4,4), padding='same'))
alex_net.add(BatchNormalization())
alex_net.add(Activation('relu'))
alex_net.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
alex_net.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
alex_net.add(BatchNormalization())
alex_net.add(Activation('relu'))
alex_net.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
alex_net.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
alex_net.add(BatchNormalization())
alex_net.add(Activation('relu'))

#4th Convolutional Layer
alex_net.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
alex_net.add(BatchNormalization())
alex_net.add(Activation('relu'))

#5th Convolutional Layer
alex_net.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
alex_net.add(BatchNormalization())
alex_net.add(Activation('relu'))
alex_net.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
alex_net.add(Flatten())
# 1st Fully Connected Layer
alex_net.add(Dense(4096, input_shape=(32,32,3,)))
alex_net.add(BatchNormalization())
alex_net.add(Activation('relu'))
# Add Dropout to prevent overfitting
alex_net.add(Dropout(0.4))

#2nd Fully Connected Layer
alex_net.add(Dense(4096))
alex_net.add(BatchNormalization())
alex_net.add(Activation('relu'))
#Add Dropout
alex_net.add(Dropout(0.4))

#3rd Fully Connected Layer
alex_net.add(Dense(1000))
alex_net.add(BatchNormalization())
alex_net.add(Activation('relu'))
#Add Dropout
alex_net.add(Dropout(0.4))

#Output Layer
alex_net.add(Dense(10))
alex_net.add(BatchNormalization())
alex_net.add(Activation('softmax'))

#Model Summary
alex_net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 8, 8, 96)          34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 96)          384       
_________________________________________________________________
activation (Activation)      (None, 8, 8, 96)          0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 96)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 256)         614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 4, 256)         0

## Train and fit the model

softmax - sed as the last activation function of a neural network to normalize the output of a network
use stochastic gradient descent as optimiser

In [2]:
# Compiling the model
alex_net.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])


In [3]:
#Keras library for CIFAR dataset
from keras.datasets import cifar10
(x_train, y_train),(x_test, y_test)=cifar10.load_data()

#Train-validation-test split
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=.3)

#Dimension of the CIFAR10 dataset
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))


#Onehot Encoding the labels.
from sklearn.utils.multiclass import unique_labels
from keras.utils import to_categorical

#Since we have 10 classes we should expect the shape[1] of y_train,y_val and y_test to change from 1 to 10
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

#Verifying the dimension after one hot encoding
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))


#Image Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1 )

val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True,zoom_range=.1)

test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip= True,zoom_range=.1)

#Fitting the augmentation defined above to the data
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))
((35000, 32, 32, 3), (35000, 10))
((15000, 32, 32, 3), (15000, 10))
((10000, 32, 32, 3), (10000, 10))


In [ ]:
xTrn_split, xTst_split, yTrn_split, yTst_split
#Dimension of the CIFAR10 dataset
#trainig data
print((xTrn_split.shape,yTrn_split.shape))

#training test data
print((xTst_split.shape,yTst_split.shape))

#test data
print((test_images_reshaped.shape,test_label_coarse_reshaped.shape))


In [4]:
#Learning Rate Annealer
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(   monitor='val_acc',   factor=.01,   patience=3,  min_lr=1e-5) 

In [5]:
#Defining the parameters
batch_size= 100
epochs=100
learn_rate=.001

In [6]:
alex_net.fit_generator(train_generator.flow(xTrn_split, yTrn_split, batch_size=batch_size), epochs = epochs, steps_per_epoch = xTrn_split.shape[0]//batch_size, validation_data = ytrntst_generator.flow(xTst_split, yTst_split, batch_size=batch_size), validation_steps = 250, callbacks = [lrr], verbose=1)


NameError: name 'xTrn_split' is not defined

##  Predict labels for the testing set and check accuracy

In [ ]:
print(alex_net.predict(test_images_reshaped))

## Plot models traing curves 

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = alex_net.evaluate(test_images_reshaped,  test_label_coarse, verbose=2)

print(test_acc)

In [ ]:
score = alex_net.evaluate(test_images_reshaped, test_label_coarse, verbose=0)
print(score)